In [9]:
import pymongo
import pandas as pd
import numpy as np
import os
import sys

In [5]:
pwd = %pwd
pwd = os.path.join(pwd, 'dados')

In [6]:
sys.path.insert(0, pwd)

In [7]:
data_mapping = {
    'EINSTEINAgosto':{'name':'EINSTEIN', 'exames':'EINSTEIN_Exames_2.csv', 'pacientes':'EINSTEIN_Pacientes_2.csv'},
    'GrupoFleuryAgosto':{'name':'GrupoFleury', 'exames':'GrupoFleury_Exames_2.csv', 'pacientes':'GrupoFleury_Pacientes_2.csv'},
    'final':{'name':'HSL', 'exames':'HSL_Exames_2.csv', 'pacientes':'HSL_Pacientes_2.csv'}
}

In [ ]:
exames = None
for key in data_mapping.keys():
    f = os.path.join(pwd, os.path.join(key, data_mapping[key]['exames']))
    
    if exames is None:
        exames = pd.read_csv(f, sep='|')
        exames['source'] = [data_mapping[key]['name']] * len(exames)
    else:
        _tmp = pd.read_csv(f, sep='|')
        _tmp['source'] = [data_mapping[key]['name']] * len(_tmp)
        exames = pd.concat([exames, _tmp], axis=0)
exames = exames.replace({np.NaN : None})

In [ ]:
pacientes = None
for key in data_mapping.keys():
    f = os.path.join(pwd, os.path.join(key, data_mapping[key]['pacientes']))
    
    if pacientes is None:
        pacientes = pd.read_csv(f, sep='|')
        pacientes['source'] = [data_mapping[key]['name']] * len(pacientes)
    else:
        _tmp = pd.read_csv(f, sep='|')
        _tmp['source'] = [data_mapping[key]['name']] * len(_tmp)
        pacientes = pd.concat([pacientes, _tmp], axis=0)
pacientes = pacientes.replace({np.NaN : None})

In [ ]:
pacientes.head()

In [ ]:
exames.head()

In [ ]:
exames = exames.to_dict(orient='records')

In [ ]:
pacientes = pacientes.to_dict(orient='records')

Create connection

In [ ]:
client = pymongo.MongoClient("mongodb+srv://admin:<password>@cluster0.pfryc.mongodb.net/<dbname>?retryWrites=true&w=majority")
client.server_info()

In [ ]:
db = client.test

Create table

In [14]:
tbl_exames = db.exams
tbl_pacientes = db.patients

In [ ]:
tbl_exames.insert_one(data[0])

In [ ]:
tbl_exames.insert_many(exames, ordered=False)

In [ ]:
tbl_pacientes.insert_many(pacientes, ordered=False)